## Cover Letter Generator using OpenAI API

In [105]:
import time
import openai
from fpdf import FPDF

#### Import Files

In [106]:
# import the API key from a file
with open("api_key.txt", "r") as file:
    api_key = file.read()

# import the prompting files
with open("template.txt", "r") as file:
    template = file.read()
with open("work_exp.txt", "r") as file:
    work_exp_content = file.read()
with open("job_description.txt", "r") as file:
    job_desc_content = file.read()

#### Build Assistant, Prompt, and Run

In [107]:
# insert key
client = OpenAI(api_key=api_key)

# building assistant
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a professional cover letter writer who specializes in highlighting applicant skills as they relate to job descriptions in under 425 words.",
    model="gpt-3.5-turbo"
)

thread = client.beta.threads.create()

# building a strong prompt
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"""
Please write an impressive cover letter for the client given the material below. Specifically, focus on the skills and experiences that directly match the job requirements, emphasizing data analysis and data science capabilities. Tailor the language to be professional, engaging, and persuasive, demonstrating a strong understanding of the role and the candidate's ability to add value to the company.

\n Here is the given material:

\n {template}

\n {work_exp_content}

\n {job_desc_content}
"""
)

# assigning run components
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# printing updating results
print("checking assistant status. ")
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        print("done!")
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        break
    else:
        print("in progress...")
        time.sleep(10)

# extracting the cover letter
for message in messages:
    if message.role == "assistant": # check if it's the assistant's response
        cover_letter_text = message.content[0].text.value
        break  # exit the loop after finding the cover letter

checking assistant status. 
in progress...
in progress...
done!


In [108]:
print(cover_letter_text)

Dear Hiring Manager,

I am writing to express my strong interest in the Data Analyst II (SQL) position at Fidelis Care. With my expertise in data analysis and data science, I believe I can contribute significantly to the company's mission of improving clinical outcomes and reducing the overall cost of care for members.

As a Data Analyst at the Computational Social Neuroscience Lab, I have gained extensive experience in executing complex data analysis projects. I have designed and implemented sophisticated R scripts, analyzing over 200 million data points to uncover insights in computational neuroscience. I have also optimized scripts for supercomputing efficiency, contributing to research with a substantial budget. My expertise in Python (Pandas, NumPy, Scikit-learn) and R (Tidyverse, ggplot2) allows me to efficiently analyze and visualize data, providing valuable insights for decision-making.

In addition, my experience as a Student Office Assistant for the Vice President of Research

#### Exporting as a Formatted PDF

In [109]:
# create a PDF object
pdf = FPDF()

# define common margin size (in centimeters)
left_margin = 25.4
right_margin = 25.4
top_margin = 25.4
bottom_margin = 25.4

# set margins
pdf.set_margins(left_margin, top_margin, right_margin)

# add a page
pdf.add_page()

# set font and font size
pdf.set_font("Times", size=12)

# write the cover letter text
pdf.write(5, cover_letter_text)

# save the PDF
pdf.output("cover_letter.pdf")

# completion message
print("Cover letter saved as cover_letter.pdf")

Cover letter saved as cover_letter.pdf
